# Yahoo ローカルサーチ API を利用した緯度経度情報の取得
探したい施設や建物の緯度・経度情報を取得します。出力されたファイルを GIS ソフトに読み込ませて使います。

### 講義室で行う場合の設定
電算機教場や研究室で行う場合には、以下の処理が必要です。

In [ ]:
import os
os.environ['HTTP_PROXY']="http://10.0.0.1:8080"
os.environ['HTTPS_PROXY']="http://10.0.0.1:8080"
os.environ['REQUESTS_CA_BUNDLE']="/Users/hsato/etc/cert_SSL-Forward-cert.crt"

### ライブラリのインストールとインポート
実行に必要なライブラリを導入します。

In [ ]:
!pip install jageocoder

In [ ]:
import requests
import time
import glob
import sys
import csv
import os
import jageocoder
import json
jageocoder.init(url='https://jageocoder.info-proto.com/jsonrpc')

### 準備
実行にあたって必要な関数などを定義します。API Key の入力を求められるので、Yahoo API key を入力して下さい。

In [ ]:
#ベースとなるURL
base_url = "https://map.yahooapis.jp/search/local/V1/localSearch"
api_key = input("API Key を入力して下さい：")
#URLパラメータ用の辞書を用意し、後からパラメータを順次格納する。01はヒット総数の確認用、02はデータ取得用。
params_01 = {"appid":api_key, "results":1, "output":"json"} 
params_02 = {"appid":api_key, "sort":"kana", "output":"json"} #繰り返し処理でID重複が生じる可能性を低減するためソート順を「かな」に設定

In [ ]:
#ヒット件数取得用の関数
def count_data(params_01):
    response_01 = requests.get(base_url, params=params_01) #ヒット件数の確認用のリクエストを投げる処理
    jsonData_01 = response_01.json()
    #print("Jsondata_01:", jsonData_01)
    time.sleep(1.0) #リクエスト１回ごとに若干時間をあけてAPI側への負荷を軽減する
    return jsonData_01["ResultInfo"]["Total"]


In [ ]:
#データ取得処理用の関数
def fetch_data(params_02, total_num, pref_name, query_word):
    max_return = 100 #APIの仕様では一回のリクエストにつき100件まで取得可能なので、その上限値を一回の取得数として設定
    pages = (int(total_num) // int(max_return)) + 1 #全件を取得するために必要なリクエスト回数を算定

    params_02['results'] = max_return #全件取得用のパラメータを設定

    Records = [] #取得データを格納するための空リストを用意

    #全件取得するためのループ処理
    for i in range(pages):
        i_startRecord = 1 + (i * int(max_return))
        params_02['start'] = i_startRecord
        response_02 = requests.get(base_url, params=params_02)

        #レスポンスのステータスが200＝正常取得だった場合の処理
        if response_02.status_code == 200:
            try:
                jsonData_02 = response_02.json() #レスポンスをJSONデータとして格納する
            except ValueError:
                print("エラー: レスポンスデータの解析処理に失敗しました。")
                sys.exit() #ここでエラーが生じた場合は処理を終了させる。ここをcontinueに変えて、この100件分だけスキップして処理続行させることも可能。
        else:
            print("エラー:", response_02.status_code)
            sys.exit() #レスポンスが正常に取得できなかった場合は処理を終了させる。

        #JSONデータ内の各要素から必要項目を指定してリストに格納する
        for poi in jsonData_02.get('Feature', []):
            #print(poi)
            #poi_id = poi.get('Id', "") #FeatureにId項目があればその値を、ない場合は空欄を返す
            poi_name = poi.get('Name', "")
            poi_property = poi.get('Property', {})
            #print(poi_property)
            poi_address = poi_property.get('Address', "")
            poi_tel = poi_property.get('Tel1', "")
            poi_station = poi_property.get('Station', [])
            #print(len(poi_station))
            #print(poi_station)
            for j in range(len(poi_station)):
                #print(poi_station[i])
                if j == 0:
                    poi_srail = poi_station[i].get('Railway', "")
                    poi_sname = poi_station[i].get('Name', "")
                    poi_sdistance = poi_station[i].get('Distance', "")
            coordinates = poi.get('Geometry', {}).get('Coordinates', "").split(",") #Coordinatesの座標値はカンマ区切りで緯度経度に分割する
            poi_lat = coordinates[1] if len(coordinates) > 1 else ""
            poi_lng = coordinates[0] if len(coordinates) > 0 else ""
            Records.append([poi_name, poi_address, poi_tel, poi_lat, poi_lng, poi_srail, poi_sname, poi_sdistance])

        sys.stdout.write(f"\r{pref_name}_{query_word}: {i+1}/{pages} is done.") #進捗状況を表示する
        sys.stdout.flush() #進捗状況を強制的に変更する
        time.sleep(1.0) #リクエスト１回ごとに若干時間をあけてAPI側への負荷を軽減する

    #CSVへの書き出し
    csv_file_path = os.path.join(f"{pref_name}_{query_word}_{total_num}.csv")
    with open(csv_file_path, 'w', newline='', encoding='cp932', errors='replace') as f:
        csvwriter = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_NONNUMERIC) #CSVの書き出し方式を適宜指定
        csvwriter.writerow(['名称', '住所', '電話番号', '緯度', '経度', '鉄道会社', '最寄り駅', '距離'])
        for record in Records:
            csvwriter.writerow(record)

    print(f"\n取得されたデータがCSV形式で出力されました。ファイル名： {csv_file_path}")

### 実行
p_area（検索エリア）と p_query（検索ワード） を設定し実行して下さい。検索結果は、「検索エリア_検索ワード_ヒット数.csv」というファイルに保存されます。

In [ ]:
p_area = "横須賀市" #検索エリア
p_query = "ラーメン" #検索ワード
p_gc = "" #検索する業種コードを入力

results = jageocoder.searchNode(p_area)
num = len(results)
if num == 1: 
    node = results[0].node
    p_ac = node.get_city_jiscode()
        
    params_01['query'] = str(p_query)
    params_02['query'] = str(p_query)
    params_01['gc'] = str(p_gc)
    params_02['gc'] = str(p_gc)
    params_01['ac'] = str(p_ac)
    params_02['ac'] = str(p_ac)

    #ヒット件数の確認用のリクエストを投げる処理
    total_num = count_data(params_01)
    print("検索結果は " + str(total_num) + "件です。")

    #ヒット件数が0件以上かつ取得条件の3100件以内だった場合は取得処理を実行、それ以外はメッセージを出して終了させる。なお、パラメータに何らかの問題があると大量のヒット件数が返されることがある。
    if total_num > 3100:
        print(f"データ取得上限の件数を超えています。検索条件を見直して下さい。")
    elif total_num > 0:
        fetch_data(params_02, total_num, p_area, p_query)
    else:
        print(f"該当するデータがありません。")
elif num >= 2:
    print("同名の地名が複数あります。もう少し詳しく指定して下さい。")
else:
    print("該当する地名はありません。")